# What is poli?

```{contents}
```

`poli` is a library for registering black box optimization functions, with a special focus on *discrete* sequence optimization. It stands for *Protein Optimization Library*, since some of the work done on drug design is done through representing proteins as discrete sequences, or sentences of amino acids.

We also build `poli-baselines` on top, allowing you to define black box optimization algorithms for discrete sequences.

These next chapters detail a basic example of how to use `poli` and `poli-baselines`. If you want to start coding now, continue to [the next chapter](./registering_an_objective_function.md)!

After these, feel free to dive deeper into how `poli` works underneath in [the chapter about the details](./diving_deeper.md).

The rest of this intro details the usual development loops we assume you'll follow when using `poli` and `poli-baselines`:

## The usual development loop

Black-box optimization algorithms inside `poli-baselines` are treated as **solvers** of **problems** defined using `poli`.

We propose to you the following process for using `poli-baselines`' optimizers, or developing your own:

### Identify the objective function

Start by identify the black-box objective function you want to optimize, and check if it's already registered in `poli`, or available in `poli`'s objective repository.

This can be done by running

In [1]:
from poli.core.registry import get_problems
print(get_problems())

['white_noise']


The output is a list of registered problems. If the function that you're interested in is not registered, you can check whether we have it in `poli`'s internal repository:

In [2]:
from poli.objective_repository import AVAILABLE_OBJECTIVES
print(AVAILABLE_OBJECTIVES)

['aloha', 'super_mario_bros', 'white_noise']


If the function isn't there, **implement it yourself!** An example of how to do this can be found in `poli_baselines/examples/00_a_simple_objective_function_registration`, or in our chapter on [registering optimization functions](./registering_an_objective_function.md).

In what follows, we will use the `white_noise` objective function. You could drop-in your own function if desired.

In [3]:
from poli import objective_factory

problem_info, f, x0, y0, _ = objective_factory.create(
    name="white_noise",
    force_register=True,
)

At this point, you can call `f` on arrays of shape `[b, L]`. In the specific case of `white_noise`, `L` can be any positive integer.

### Using a solver, or creating your own

`poli-baselines` also comes with black-box optimizers out-of-the-box. You can find them inside the library.

For example, let's use the `RandomMutation` solver, which takes the initial `x0` and randomly mutates it according to the alphabet provided in `problem_info`.

In [13]:
from poli_baselines.solvers.simple.random_mutation import RandomMutation

solver = RandomMutation(
    black_box=f,
    x0=x0,
    y0=y0,
    alphabet=problem_info.alphabet
)

print(f"x0: {x0}")
print(f"y0: {y0}")

x0: [['1' '2' '3']]
y0: [[1.58015034]]


Solvers implement a `next_candidate()` method, based on their history:

In [14]:
print(solver.history)
solver.next_candidate()

{'x': [array([['1', '2', '3']], dtype='<U1')], 'y': [array([[1.58015034]])]}


array([['1', '3', '3']], dtype='<U1')

`RandomMutation` simply selects one token at random from the alphabet:

In [ ]:
[alphabet_symbol for alphabet_symbol in solver.alphabet.keys()]

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

**If you are interested in building your own solver**, check out [the chapter detailing how `RandomMutation` is implemented](./defining_a_problem_solver.md).

### Optimizing

Once you have a black box objective function `f` and a solver on top, the optimization is quite easy:

In [19]:
solver.solve(max_iter=100)

In [20]:
print(solver.get_best_solution())

[['0' '2' '3']]


Of course, this example is trivial. We dive deeper in the next chapters.

## Conclusion

This chapter discusses the usual development loop using `poli` and `poli-baselines`:
1. Start by identifying/building your objective function,
2. continue by creating/using a solver in `poli_baselines`, and
3. use the `solve` method to run a number of iterations from the solver.

The next three chapters talk about another trivial example, diving deeper in the process of defining your own objective functions and solvers. You can continue there, or by checking [the currently implemented repository of objective functions inside `poli` TODO: ADD]().